In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import openpyxl
import requests

In [8]:
curl = f"https://contract.mexc.com/api/v1/contract/detail/"
response = requests.get(curl)
data = pd.DataFrame(response.json()['data'])
result = pd.DataFrame()
result['币对'] = data['symbol'].astype(str)
result[['position_limit_min', 'max_leverage', 'position_limit_max']] = pd.DataFrame([[None, None, None]], index=result.index)

In [9]:
# reject_cookies
def reject_cookies(driver, accept_xpath):
    try:
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, accept_xpath))
        )

        accept_button.click()
    except Exception as e:
        print(f"An error occurred while accepting cookies: {e}")

reject_xpath = "/html/body/div[2]/div[2]/div/div[1]/div/div[2]/div/button[2]"

In [10]:
# click and input currencies
def click_input(driver, click_xpath, input_xpath, test_xpath, target_xpath ,result):
    # click
    click_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, click_xpath))
        )
    click_button.click()
    # input
    input_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, input_xpath))
    )
    for currency in result["币对"]:
        currency1 = currency.replace("_","")
        input_box.clear()
        input_box.send_keys(currency1)
        input_box.send_keys(Keys.RETURN)
        # check whether the result matches the input
        test = driver.find_element(By.XPATH,test_xpath)
        value = test.get_attribute("value")
        value = str(value)
        if value == currency1:
            # extract
            position_limit_max = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, target_xpath["position_max_xpath"]))
            )
            max_leverage = driver.find_element(By.XPATH, target_xpath["max_leverage_xpath"])
            position_limit_min = driver.find_element(By.XPATH, target_xpath["position_min_xpath"])

            position_limit_max = position_limit_max.text
            max_parts = position_limit_max.split('-')
            position_limit_max = max_parts[1].strip()

            max_leverage = max_leverage.text
            position_limit_min = position_limit_min.text
            min_parts = position_limit_min.split('-')
            position_limit_min = min_parts[1].strip()
            # load
            result.loc[result["币对"]==currency,"position_limit_min"]=position_limit_min
            result.loc[result["币对"]==currency,"max_leverage"]=max_leverage
            result.loc[result["币对"]==currency,"position_limit_max"]=position_limit_max
        else: 
            result.loc[result["币对"]==currency,"position_limit_min"]='NA'
            result.loc[result["币对"]==currency,"max_leverage"]='NA'
            result.loc[result["币对"]==currency,"position_limit_max"]='NA'
        time.sleep(1)
        click_button.click()

click_xpath = "/html/body/div[1]/div[3]/div[1]/div[2]/div[3]/div/div[2]/div[1]/div/div[1]"
input_xpath = "//html/body/div[1]/div[3]/div[1]/div[2]/div[3]/div/div[2]/div[1]/div/div[2]/div/div/input"
test_xpath = "/html/body/div[1]/div[3]/div[1]/div[2]/div[3]/div/div[2]/div[1]/div/div[1]/input"
target_xpath = {
    "position_max_xpath" : "/html/body/div[1]/div[3]/div[1]/div[2]/div[3]/div/div[3]/div/div/div/div/table/tbody/tr[1]/td[2]",
    "max_leverage_xpath" : "/html/body/div[1]/div[3]/div[1]/div[2]/div[3]/div/div[3]/div/div/div/div/table/tbody/tr[1]/td[3]",
    "position_min_xpath" : "/html/body/div[1]/div[3]/div[1]/div[2]/div[3]/div/div[3]/div/div/div/div/table/tbody/tr[last()]/td[2]"
}

In [11]:
def main(url):
    driver = webdriver.Chrome()   
    try:
        driver.get(url)
        reject_cookies(driver, reject_xpath)
        click_input(driver, click_xpath, input_xpath, test_xpath, target_xpath, result)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit() 

In [12]:
url = "https://www.binance.com/en/futures/trading-rules/perpetual/leverage-margin"
main(url)

An error occurred: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.218)
Stacktrace:
	GetHandleVerifier [0x00007FF68C6E2142+3514994]
	(No symbol) [0x00007FF68C300CE2]
	(No symbol) [0x00007FF68C1A76AA]
	(No symbol) [0x00007FF68C180AFD]
	(No symbol) [0x00007FF68C21CB1B]
	(No symbol) [0x00007FF68C23218F]
	(No symbol) [0x00007FF68C215D93]
	(No symbol) [0x00007FF68C1E4BDC]
	(No symbol) [0x00007FF68C1E5C64]
	GetHandleVerifier [0x00007FF68C70E16B+3695259]
	GetHandleVerifier [0x00007FF68C766737+4057191]
	GetHandleVerifier [0x00007FF68C75E4E3+4023827]
	GetHandleVerifier [0x00007FF68C4304F9+689705]
	(No symbol) [0x00007FF68C30C048]
	(No symbol) [0x00007FF68C308044]
	(No symbol) [0x00007FF68C3081C9]
	(No symbol) [0x00007FF68C2F88C4]
	(No symbol) [0x00007FF9892026AD]
	RtlUserThreadStart [0x00007FF989DCA9F8+40]

